# Phone

In [1]:
import util

1000


The util.py file contains a couple useful methods and packages. It contains the entire socket package, the entire Cryptodome package, as well as the following methods: 
- encrypt(message, key, nonce)
- decrypt(ciphertext, key, nonce)
- hash(message, key, nonce)
- generate_n()
- generate_k()
These methods all use various Cryptodome features, with several constants to ensure we're building based on the same model design.

In [10]:
k_ma = util.generate_k()
n = util.generate_n()
req = b"thisisamessage"

h = util.hash(n, req, k_ma)
m = util.encrypt(req, k_ma, n)

client_socket = util.socket.socket(util.socket.AF_INET, util.socket.SOCK_STREAM)
client_socket.connect((util.SERVER["alias"], util.SERVER["port"]))

print(h)
print(m)

client_socket.sendall(n + m + h)

client_socket.close()